In [1]:
import pygame
import moderngl
import numpy as np
import glm
from LoadObject import getObjectData
from Assignment6Specs import scaleFactors, angles, tVectors

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame
pygame.init()

# Set OpenGL version and create an OpenGL context
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)

# Get the ModernGL context
gl = moderngl.create_context()
gl.enable(moderngl.DEPTH_TEST)

# Initialize window size variables
window_width, window_height = 800, 600


2024-10-02 13:36:34.123 Python[16606:5770739] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [3]:
# Shader programs
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec3 in_position;
layout (location = 1) in vec3 in_normal;

uniform mat4 view;
uniform mat4 perspective;

in mat4 in_modelMat;  // Instance model matrix
out vec3 f_normal;

void main() {
    f_normal = in_normal;
    gl_Position = perspective * view * in_modelMat * vec4(in_position, 1.0);
}
'''

In [4]:
fragment_shader_code = '''
#version 330 core

in vec3 f_normal;
out vec4 out_color;

void main() {
    out_color = vec4(0.5 * (normalize(f_normal) + 1.0), 1.0);
}
'''

In [5]:
# Compile shaders and link them into a program
shaderProgram = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

In [6]:
# Function to load object data and create vertex buffer
def load_object(obj_file):
    vertexData, bounds = getObjectData(obj_file, normal=True)
    vertexBuffer = gl.buffer(vertexData.astype('f4').tobytes())
    return vertexBuffer, bounds

In [7]:
# Load the cube (floor) and teapot object
cube_vertex_buffer, cube_bounds = load_object("Hexahedron.obj")
teapot_vertex_buffer, teapot_bounds = load_object("Teapot.obj")

Normal computed.
Normal exists


In [8]:
# Prepare the rectangular floor by scaling and translating the cube
floor_model_matrix = glm.translate(glm.mat4(1), glm.vec3(0, -1, 0)) * glm.scale(glm.mat4(1), glm.vec3(10, 0.05, 10))
floor_matrix_buffer = gl.buffer(np.array(list(floor_model_matrix)).astype('float32').tobytes())

# Prepare the teapot model matrices for 100 instances
teapot_matrices = []
for i in range(100):
    scale_matrix = glm.scale(glm.mat4(1), glm.vec3(scaleFactors[i]))
    rotate_matrix = glm.rotate(glm.mat4(1), glm.radians(angles[i]), glm.vec3(0, 1, 0))

    # Adjust the Y-axis translation so teapots sit on top of the cube
    adjusted_tVector = glm.vec3(tVectors[i])

    # Approximate teapot height using bounds.radius
    teapot_height = teapot_bounds.radius * 2.0  # Assuming radius is half the height of the teapot
    adjusted_tVector.y -= 0.95 # Remove the space between the teapots and the cube

    translate_matrix = glm.translate(glm.mat4(1), adjusted_tVector)
    model_matrix = translate_matrix * rotate_matrix * scale_matrix
    teapot_matrices.append(list(model_matrix))

# Create buffer for the teapot model matrices
teapot_transformation_buffer = gl.buffer(np.array(teapot_matrices).astype('float32').tobytes())

In [9]:
# Pygame window setup
pygame.display.set_caption("Assignment 6: Nelson Herrera")

# Create the teapot renderable with instanced model matrix attribute
teapot_renderable = gl.vertex_array(
    shaderProgram,
    [
        (teapot_vertex_buffer, '3f 3f /v', 'in_position', 'in_normal'),
        (teapot_transformation_buffer, '16f /i', 'in_modelMat')
    ]
)

# Create the cube (floor) renderable
floor_renderable = gl.vertex_array(
    shaderProgram,
    [
        (cube_vertex_buffer, '3f 3f /v', 'in_position', 'in_normal'),
        (floor_matrix_buffer, '16f /i', 'in_modelMat')
    ]
)

# Set the camera parameters
camera_position = glm.vec3(20, 10, 0)
camera_target = glm.vec3(0, 0, 0)
camera_up = glm.vec3(0, 1, 0)

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            running = False
        
        # Handle window resize event
        if event.type == pygame.VIDEORESIZE:
            window_width, window_height = event.w, event.h
            pygame.display.set_mode((window_width, window_height), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
            gl.viewport = (0, 0, window_width, window_height)

    # Clear the screen
    gl.clear(0.2, 0.2, 0.2)
    
    # Compute aspect ratio
    aspect_ratio = window_width / window_height

    # Update the orbiting camera
    angle_in_radian = pygame.time.get_ticks() / 2000.0
    rotation_matrix = glm.rotate(glm.mat4(1), angle_in_radian, glm.vec3(0, 1, 0))
    rotated_camera_position = glm.vec3(rotation_matrix * glm.vec4(camera_position, 1.0))
    view_matrix = glm.lookAt(rotated_camera_position, camera_target, camera_up)
    
    # Compute the perspective projection matrix
    near, far, fov = 0.1, 100, 60
    perspective_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near, far)

    # Upload matrices to the shader
    shaderProgram['view'].write(view_matrix)
    shaderProgram['perspective'].write(perspective_matrix)

    # Render the floor (cube)
    floor_renderable.render(moderngl.TRIANGLES)

    # Render 100 teapots in one call, passing the number of instances
    teapot_renderable.render(moderngl.TRIANGLES, instances=100)

    # Swap buffers
    pygame.display.flip()

# Cleanup
pygame.quit()
quit()